## DSANIB: A novel framework for drug-target interaction prediction by dual-view synergistic attention network with information bottleneck

####Import required modules.

In [ ]:
from models import DSANIB
from time import time
from utils import set_seed, graph_collate_func, mkdir
from configs import get_cfg_defaults
from dataProcess import DTIDataset
from torch.utils.data import DataLoader
from train import Training
import torch
import argparse
import os
import pandas as pd

####Configuration.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg_path = "./configs/DSANIB_Demo.yaml"
data = "bindingdb_sample"

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()

torch.cuda.empty_cache()
warnings.filterwarnings("ignore")
set_seed(cfg.SETUP.SEED)
mkdir(cfg.RESULT.OUTPUT_PATH)
print(f"Config yaml: {cfg_path}")
print(f"Running on: {device}")
print(f"Hyperparameters:")
dict(cfg)

####Data Loader

In [ ]:
dataFolder = f'./datasets/{data}'

train_path = os.path.join(dataFolder, 'train.csv')
val_path = os.path.join(dataFolder, "val.csv")
test_path = os.path.join(dataFolder, "test.csv")
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

train_dataset = DTIDataset(df_train.index.values, df_train)
val_dataset = DTIDataset(df_val.index.values, df_val)
test_dataset = DTIDataset(df_test.index.values, df_test)

params = {'batch_size': cfg.SETUP.BATCH_SIZE, 'shuffle': True, 'num_workers': cfg.SETUP.NUM_WORKERS,
          'drop_last': True, 'collate_fn': graph_collate_func}

train_data = DataLoader(train_dataset, **params)
params['shuffle'] = False
params['drop_last'] = False

val_data = DataLoader(val_dataset, **params)
test_data = DataLoader(test_dataset, **params)

####Setup Model and Optimizer

In [ ]:
model = DSANIB(**cfg).to(device)
opt = torch.optim.Adam(model.parameters(), lr=cfg.SETUP.LR)
if torch.cuda.is_available():
  torch.backends.cudnn.benchmark = True

####Model Training and Test Optimized Model

In [ ]:
trainer = Training(model, opt, device, train_data, val_data, test_data, opt_da=None **cfg)
    result = trainer.train()
with open(os.path.join(cfg.RESULT.OUTPUT_DIR, "model_architecture.txt"), "w") as wf:
    wf.write(str(model))
print(f"Directory for saving result: {cfg.RESULT.OUTPUT_PATH}")